In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import PIL.Image as Image
import torch.nn as nn
import torchvision
import torch.optim as optim
import sys
from captum.attr import GuidedGradCam
import cv2
from PIL import Image
from matplotlib.colors import LinearSegmentedColormap
from captum.attr import visualization as viz

sys.path.insert(0, '../src')
from bird_dataset import *
from XAI_birds_dataloader import *
from tqdm import tqdm
from models.multi_task_model import *
from XAI_birds_dataloader import *
from XAI_BirdAttribute_dataloader import *

In [7]:
# #hide
# from fastai.vision.all import *
# from fastai.text.all import *
# from fastai.collab import *
# from fastai.tabular.all import *

In [16]:
bd = BirdDataset(preload=True, attr_file='attributes-subset')

In [17]:
# class Bird_Attribute_Loader(XAI_Birds_Dataset):
#     '''
#     Loads in x amount of attributes in bd.attributes
#     self.attrs:str or list -- loads in x attributes into a list of labels for Pytorch
#     '''
#     def __init__(self, bd:BirdDataset, attrs, subset=True, transform=None, train=True, val=False, random_seed=42):
#         XAI_Birds_Dataset.__init__(self, bd, subset=subset, transform=transform, train=train, val=val, random_seed=random_seed)
#         self.attrs = attrs
#         self.class_dict = self._set_classes_attributes()
#         self.images, self.attr_indices = self._filter_images_by_attributes()
#         print(f'num_images: {len(self.images)}')
        

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
#         img_path = os.path.join(self.bd.img_dir, self.images[idx]['filepath'])
#         image = Image.open(img_path)
#         if isinstance(self.attrs, str):
#             attr = self.images[idx]['attributes'][self.attr_indices[idx]]
#             label = self.class_dict[attr]
#             sample = {'image': image, 'label':label}
#         elif isinstance(self.attrs, list):
#             attrs = sorted([self.images[idx]['attributes'][i] for i in self.attr_indices[idx]])
# #             print(attrs)
# #             print("ATTRIBUTES TO INDEX:",attrs)
#             labels = [self.class_dict[attr.split('::')[0]][attr] for attr in attrs]
#             sample = {'image': image, 'labels':labels}
#         if self.transform:
#             sample['image'] = self.transform(sample['image'])
#         return sample
    
#     def _set_classes_attributes(self):
#         pd_attr = pd.Series(self.bd.attributes)
#         if isinstance(self.attrs, str):
#             attrs_dict = pd_attr[pd_attr.str.contains(self.attrs)].to_dict()
#             class_dict = dict(zip(attrs_dict.values(), range(len(attrs_dict))))
#         elif isinstance(self.attrs, list):
#             attrs_dict = dict()
#             for attribute in self.attrs:
#                 attr_dict = pd_attr[pd_attr.str.contains(attribute)].to_dict()
#                 attrs_dict[attribute] = dict(zip(attr_dict.values(), range(len(attr_dict))))
# #             print(f'ATTRS DICT: {attrs_dict}')
# #             class_dict = dict(zip(attrs_dict.values(), range(len(attrs_dict))))
#         return attrs_dict
    
#     def _filter_images_by_attributes(self):
#         filt_images = []
#         attr_indices = []
#         for img in self.images:
#             check=0
#             attr_index = []
#             attrs = []
            
#             for idx, attr in enumerate(img['attributes']):
#                 if isinstance(self.attrs, str):
#                     if self.attrs in attr:
#                         filt_images.append(img)
#                         attr_indices.append(idx)
#                         break
#                 elif isinstance(self.attrs, list):
#                     for attribute in self.attrs:
#                         if attribute in attr:
#                             check+=1
# #                             print(attribute)
#                             attr_index.append(idx)
#                             attrs.append(attribute)
#                 else: raise(ValueError, "self.attrs must be a string or a list of strings")
#             if check==len(self.attrs): # only append to images/indices if all attributes in self.attrs are in the images attributes
#                 unique_attrs = [attr.split('::')[0] for attr in attrs]
# #                 print(unique_attrs)
#                 if len(set(unique_attrs)) == len(self.attrs):
# #                     if img not in filt_images:
#                     filt_images.append(img)
#     #                     else: pass
#     #                             print('img already herre')
#     #                         print('wowie')
#                     attr_indices.append((list(attr_index)))
# #                 else: print("Attributes aren't unique:",attr)
#         return filt_images, attr_indices

In [18]:
vgg16 = models.vgg16_bn(pretrained=True)


In [19]:
# vgg16

In [20]:
# bd.images

In [21]:
# [bd.images[i]['attributes'] for i in list(bd.images.keys()) if 'has_bill_shape' in bd.images[i]['attributes']]

In [44]:
bd.images

{484: {'filepath': '010.Red_winged_Blackbird/Red_Winged_Blackbird_0091_4096.jpg',
  'class_label': 10,
  'bounding_box': [187.0, 129.0, 131.0, 226.0],
  'parts': {'back': [256.0, 197.0],
   'beak': [203.0, 137.0],
   'breast': [213.0, 196.0],
   'crown': [230.0, 142.0],
   'forehead': [223.0, 137.0],
   'left eye': [218.0, 142.0],
   'left leg': [247.0, 287.0],
   'left wing': [231.0, 215.0],
   'nape': [244.0, 172.0],
   'right leg': [239.0, 268.0],
   'tail': [296.0, 319.0],
   'throat': [214.0, 154.0]},
  'attributes': ['has_bill_shape::dagger',
   'has_wing_color::yellow',
   'has_wing_color::black',
   'has_wing_color::white',
   'has_wing_color::red',
   'has_upperparts_color::black',
   'has_underparts_color::black',
   'has_breast_pattern::solid',
   'has_back_color::black',
   'has_tail_shape::fan-shaped_tail',
   'has_upper_tail_color::black',
   'has_head_pattern::plain',
   'has_breast_color::black',
   'has_throat_color::black',
   'has_eye_color::black',
   'has_bill_leng

In [45]:
trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
train_bird_dataset = Bird_Attribute_Loader(bd, attrs=['has_breast_pattern','has_bill_shape'], species=True, transform=trans, train=True)
val_bird_dataset = Bird_Attribute_Loader(bd, attrs=['has_breast_pattern','has_bill_shape'], species=True, transform=trans, train=False, val=True)

In [46]:
# bd.images

In [47]:
train_bird_dataset[0]

KeyError: 'has_breast_pattern::solid'

In [10]:
# train_bird_dataset[22]

In [11]:
# label0 = []
# label1 = []
# for idx, data in enumerate(train_bird_dataset):
# #     print(label)
# #     break
#     if data['labels'][0] == 11:
#         print(idx)
#         print(data)
#         break
#     label0.append(data['labels'][0])
#     label1.append(data['labels'][1])

In [12]:
# pd.Series(label0).value_counts()

In [13]:
# data['labels']

In [14]:
# pd.Series(label1).value_counts()

In [15]:
# len(train_bird_dataset)
# len(val_bird_dataset)

In [16]:
# for sample in train_bird_dataset:
#     print(sample)
#     break

In [24]:
class MultiTaskModel(nn.Module):
    """
    Creates a MTL model for the two attributes: 'has_bill_shape' and 'has_wing_color'
    """
    def __init__(self, model,ps=0.5):
        super(MultiTaskModel,self).__init__()
        
#         num_feats = model.classifier[6].in_features
#         features = list(model.classifier.children())[:-1]
#         features.extend([nn.Linear(num_feats, len(train_bird_dataset.class_dict))])
#         vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
        
        self.encoder = model        #fastai function that creates an encoder given an architecture
        
        self.fc1 = nn.Linear(1000, 9)    
        self.fc2 = nn.Linear(1000, 15)

    def forward(self,x):

#         x = nn.ReLU(self.encoder(x))
        x = self.encoder(x)
        
        bill_shape = self.fc1(x)
        wing_color = self.fc2(x)

        return bill_shape, wing_color

In [30]:
class MultiTaskLossWrapper(nn.Module):
    '''
    Multi-Task loss for two attributes only
    '''
    def __init__(self):
        super(MultiTaskLossWrapper, self).__init__()
#         self.task_num = task_num
#         self.log_vars = nn.Parameter(torch.zeros((task_num)))

    def forward(self, preds, labels):

#         print("PREDICTIONS:",preds[0])
#         print("LABELS:",labels[0])
        loss = sum([nn.CrossEntropyLoss()(preds[i].reshape(1, -1), labels[i].reshape(1)) for i in range(len(preds))])
        
#         loss0 = nn.CrossEntropyLoss()(preds[0].reshape(1, -1), labels[0].reshape(1))
#         loss1 = nn.CrossEntropyLoss()(preds[1].reshape(1, -1), labels[1].reshape(1))
        return loss

In [31]:
# train_bird_dataset[0]

In [32]:
model = MultiTaskModel(vgg16)

In [33]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# If 'cuda:0' is printed, it means GPU is available.

if torch.cuda.is_available():
    model.cuda()

batch_size = 1
trainloader = DataLoader(train_bird_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_bird_dataset, batch_size=batch_size, shuffle=True)

loss_func = MultiTaskLossWrapper().to(device)
opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

cuda:0


In [35]:
data = train_bird_dataset[0]
inputs, labels = data['image'], torch.LongTensor(data['labels'])

In [42]:
# model(inputs)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 64 3 3, but got 3-dimensional input of size [3, 224, 224] instead

In [38]:
labels

tensor([188,   1,  11])

In [36]:
loss_func(inputs, labels)

tensor(33.5299)

In [22]:
%%time
avg_losses = []
avg_val_losses = []
epochs = 50
print_freq = 100
val_acc = []
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs.
#         print("LABELS:",data['labels'])
        inputs, labels = data['image'], torch.LongTensor(data['labels'])

        # Move the inputs to the specified device.
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients.
        opt.zero_grad()

        # Forward step.
        outputs = model(inputs)
#         print("OUTPUTS:",outputs) 
#         break
        loss = loss_func(outputs, labels)
#         print(loss)
        # Backward step.
        loss.backward()
        
        # Optimization step (update the parameters).
        opt.step()

        # Print statistics.
        running_loss += loss.item()
#         print('running loss', running_loss)
#         print(outputs)
        if i % print_freq == print_freq - 1: # Print every several mini-batches.
            avg_loss = running_loss / print_freq
            print('[epoch: {}, i: {:5d}] avg mini-batch loss: {:.3f}'.format(
                epoch, i, avg_loss))
            running_loss = 0.0
            
    avg_losses.append(avg_loss)
    
    model.eval()
    with torch.no_grad():
        num_correct=0
        val_losses = []
        data_iter = iter(valloader)
        for val_data in data_iter:
            val_inputs, val_labels = val_data['image'].cuda(), torch.LongTensor(val_data['labels']).cuda()
#                     print('val inputs: ',val_inputs.size())
#                     print('val labels: ',val_labels)
#                     print('inputs: ',inputs.size())
#             print(val_labels)
            val_outputs = model(val_inputs)
#                     print('val outputs: ',val_outputs)
#                     print('val loss: ',nn.CrossEntropyLoss()(val_outputs, val_labels))
            opt.zero_grad() #zero the parameter gradients
            _, val_predicted = [torch.max(i, 1) for i in val_outputs]
            #     print(predicted)
            # print(labels)
            # print(predicted)
#             num_correct_k += topk_accuracy(k, labels, outputs)
            num_correct += sum(np.array(val_labels.cpu())==np.array(val_predicted))
#             print(predicted)
#             print(val_labels)
            val_losses.append(loss_func(val_outputs, val_labels).item())
        acc = num_correct/(len(data_iter)*batch_size)
        val_acc.append(acc)
        print('Validation accuracy:',acc)
        print('Average validation loss:',np.mean(val_losses))
        avg_val_losses.append(np.mean(val_losses))
    model.train()
print('Finished Training.')

[epoch: 0, i:    99] avg mini-batch loss: 3.912
[epoch: 0, i:   199] avg mini-batch loss: 3.614
[epoch: 0, i:   299] avg mini-batch loss: 3.426
Validation accuracy: 0.52
Average validation loss: 2.9852547895908357
[epoch: 1, i:    99] avg mini-batch loss: 3.237
[epoch: 1, i:   199] avg mini-batch loss: 3.150
[epoch: 1, i:   299] avg mini-batch loss: 3.132
Validation accuracy: 0.5
Average validation loss: 3.179421987533569
[epoch: 2, i:    99] avg mini-batch loss: 3.223
[epoch: 2, i:   199] avg mini-batch loss: 3.175
[epoch: 2, i:   299] avg mini-batch loss: 2.992
Validation accuracy: 0.54
Average validation loss: 3.096922662258148
[epoch: 3, i:    99] avg mini-batch loss: 3.151
[epoch: 3, i:   199] avg mini-batch loss: 2.797
[epoch: 3, i:   299] avg mini-batch loss: 2.940
Validation accuracy: 0.55
Average validation loss: 3.0018569660186767
[epoch: 4, i:    99] avg mini-batch loss: 2.828
[epoch: 4, i:   199] avg mini-batch loss: 2.884
[epoch: 4, i:   299] avg mini-batch loss: 2.860
Val

[epoch: 38, i:    99] avg mini-batch loss: 0.013
[epoch: 38, i:   199] avg mini-batch loss: 0.001
[epoch: 38, i:   299] avg mini-batch loss: 0.002
Validation accuracy: 0.65
Average validation loss: 3.293736689090729
[epoch: 39, i:    99] avg mini-batch loss: 0.001
[epoch: 39, i:   199] avg mini-batch loss: 0.001
[epoch: 39, i:   299] avg mini-batch loss: 0.001
Validation accuracy: 0.7
Average validation loss: 3.372641153335571
[epoch: 40, i:    99] avg mini-batch loss: 0.000
[epoch: 40, i:   199] avg mini-batch loss: 0.001
[epoch: 40, i:   299] avg mini-batch loss: 0.005
Validation accuracy: 0.69
Average validation loss: 3.130922033786774
[epoch: 41, i:    99] avg mini-batch loss: 0.001
[epoch: 41, i:   199] avg mini-batch loss: 0.002
[epoch: 41, i:   299] avg mini-batch loss: 0.001
Validation accuracy: 0.65
Average validation loss: 3.245144898891449
[epoch: 42, i:    99] avg mini-batch loss: 0.001
[epoch: 42, i:   199] avg mini-batch loss: 0.000
[epoch: 42, i:   299] avg mini-batch lo

In [25]:
torch.save(model.state_dict(), '../models/transfer_multi-task_2_50_epoch_state_dict.pth')

In [26]:
(avg_losses)

[0.52,
 0.5,
 0.54,
 0.55,
 0.55,
 0.51,
 0.5,
 0.56,
 0.45,
 0.53,
 0.52,
 0.55,
 0.62,
 0.58,
 0.59,
 0.58,
 0.58,
 0.56,
 0.52,
 0.6,
 0.54,
 0.56,
 0.34,
 0.47,
 0.54,
 0.62,
 0.62,
 0.53,
 0.65,
 0.61,
 0.54,
 0.55,
 0.66,
 0.71,
 0.68,
 0.65,
 0.63,
 0.64,
 0.65,
 0.7,
 0.69,
 0.65,
 0.66,
 0.66,
 0.55,
 0.68,
 0.69,
 0.67,
 0.65,
 0.65]

In [40]:
labels[0]

tensor(7, device='cuda:0')

In [44]:
model(inputs)

(tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
        grad_fn=<AddmmBackward>),
 tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
        device='cuda:0', grad_fn=<AddmmBackward>))

In [45]:
model

MultiTaskModel(
  (encoder): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, paddin

In [42]:
outputs

(tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
        grad_fn=<AddmmBackward>),
 tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
        device='cuda:0', grad_fn=<AddmmBackward>))

In [41]:
outputs[0].reshape(1, -1)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<AsStridedBackward>)

In [125]:
labels[0]

tensor(6., device='cuda:0')

In [128]:
outputs[0].size()

torch.Size([1, 9])

In [131]:
labels[0]

tensor(11., device='cuda:0')

In [129]:
outputs[1].size()

torch.Size([1, 15])

In [126]:
nn.CrossEntropyLoss()(outputs, labels)

AttributeError: 'tuple' object has no attribute 'log_softmax'